In [4]:
import json
from pathlib import Path

import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

In [2]:
root_dir = Path().resolve().parent
cache_dir = root_dir / ".cache" / "test"

In [10]:
with open(cache_dir / "out.json", "r") as f:
    output = json.load(f)


scene_data = output["data"]["scene_data"]["scenes"]
image_data = output["data"]["thumbnail_data"]

records = []
for i in range(len(image_data)):
    i_image_data = image_data[i]
    images = i_image_data["images"]
    not_images = {k: v for k, v in i_image_data.items() if k != "images"}

    for image in images:
        record = not_images.copy()
        record.update(image)
        records.append(record)

df_scenes = pd.DataFrame(scene_data)
df_images = pd.DataFrame(records)

In [11]:
df_scenes.head()

,title,content,telop,image_search_query,start_sec,end_sec,process_index
0,Scene 1 導入：海と夜景が日常の街,海と観覧車の夜景、レトロな街並み、ディープな飲み屋街までそろうエリアを、住む目線でまとめて紹...,海と夜景が日常の街,横浜 みなとみらい 夜景 街並み,0.0,1.5,0
1,Scene 2 駅名紹介：横浜駅,今回紹介するのは、JR東海道本線や湘南新宿ラインなどが使える『横浜駅』エリアです。,今日の住みたい駅は横浜駅,横浜駅 東口 西口 駅前 俯瞰,1.5,3.0,1
2,Scene 3 全体像：海も繁華街も近い街,横浜駅を起点に、海辺の高層ビル街からレトロな飲み屋街までが徒歩や1〜2駅圏内に集まり、エリア...,海も繁華街も揃う横浜生活,横浜駅 周辺地図 みなとみらい 野毛 中華街 俯瞰,3.0,4.5,2
3,Scene 4 アクセス：都心への通勤・通学,横浜駅からは東京駅へ約25分、新宿駅へ約31分、品川駅へ約12分と、主要ターミナルに乗換なし...,東京25分・新宿31分圏内,JR東海道線 湘南新宿ライン 車内 路線図 横浜から東京 新宿,4.5,6.5,3
4,Scene 5 海辺カフェ＆夜景エリア,みなとみらいや東口ポートサイド周辺には、海を望むテラスカフェやスイーツ、ホテルラウンジが集ま...,海辺カフェと夜景スポット充実,横浜 みなとみらい 海辺カフェ テラス 夜景 ランドマークタワー,6.5,8.5,4


In [12]:
df_images.head()

,process_index,query,id,snippet,link,thumbnailLink,width,height,rank
0,0,横浜駅 海と観覧車 夜景 住宅街,img_1,横浜在住歴20年、おしゃれと港だけじゃない。横浜の住みやすさとエリア ...,https://www.housecom.jp/kurashiate/images/2022...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1999,1333,1
1,0,横浜駅 海と観覧車 夜景 住宅街,img_2,横浜の夜景スポット26選！デートや撮影におすすめの場所がわかる,https://nightscape.tokyo/wp-content/uploads/20...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1300,866,2
2,0,横浜駅 海と観覧車 夜景 住宅街,img_3,横浜で夜景がきれいに見えるスポット28選 | TABIPPO.NET,https://tabippo.net/wp-content/uploads/2017082...,https://encrypted-tbn0.gstatic.com/images?q=tb...,680,453,3
3,0,横浜駅 海と観覧車 夜景 住宅街,img_4,神奈川県の夜景スポット記事の一覧 | 夜景FANマガジン,https://yakei-fan.com/images/magazine/yakei/pi...,https://encrypted-tbn0.gstatic.com/images?q=tb...,1200,800,4
4,0,横浜駅 海と観覧車 夜景 住宅街,img_5,横浜ベイホテル東急客室から見るみなとみらいの夜景】 タイムラプスで ...,https://lookaside.instagram.com/seo/google_wid...,https://encrypted-tbn0.gstatic.com/images?q=tb...,720,720,5


In [13]:
sa_filepath = root_dir / "sns_ai_automation_agency/sa-key.json"

# 認証情報の設定
scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

# サービスアカウントキーのパスを指定
creds = Credentials.from_service_account_file(sa_filepath, scopes=scopes)

# gspreadクライアントの初期化
client = gspread.authorize(creds)

In [14]:
sheet_url = "https://docs.google.com/spreadsheets/d/1bzROAf4BHIdq551QpWbRL1xRnB2hFqJGQ9Mu9Vj0w-c/edit?usp=sharing"

In [16]:
# スプレッドシートを開く
sheet = client.open_by_url(sheet_url)

worksheet = sheet.worksheet("scenes")
set_with_dataframe(worksheet, df_scenes)

worksheet = sheet.worksheet("images")
set_with_dataframe(worksheet, df_images)